In [2]:
import pandas as pd
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
train.head()

,id,product_uid,product_title,search_term,relevance
0,2,100001,Simpson Strong-Tie 12-Gauge Angle,angle bracket,3.00
1,3,100001,Simpson Strong-Tie 12-Gauge Angle,l bracket,2.50
2,9,100002,BEHR Premium Textured DeckOver 1-gal. #SC-141 ...,deck over,3.00
3,16,100005,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,rain shower head,2.33
4,17,100005,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,shower only faucet,2.67


In [4]:
test.head()

,id,product_uid,product_title,search_term
0,1,100001,Simpson Strong-Tie 12-Gauge Angle,90 degree bracket
1,4,100001,Simpson Strong-Tie 12-Gauge Angle,metal l brackets
2,5,100001,Simpson Strong-Tie 12-Gauge Angle,simpson sku able
3,6,100001,Simpson Strong-Tie 12-Gauge Angle,simpson strong ties
4,7,100001,Simpson Strong-Tie 12-Gauge Angle,simpson strong tie hcc668


In [5]:
train.describe()

,id,product_uid,relevance
count,74067.000000,74067.000000,74067.000000
mean,112385.709223,142331.911553,2.381634
std,64016.573650,30770.774864,0.533984
min,2.000000,100001.000000,1.000000
25%,57163.500000,115128.500000,2.000000
50%,113228.000000,137334.000000,2.330000
75%,168275.500000,166883.500000,3.000000
max,221473.000000,206650.000000,3.000000


In [7]:
test.describe()

,id,product_uid
count,166693.000000,166693.000000
mean,123932.839741,149713.170445
std,71518.389174,37270.476881
min,1.000000,100001.000000
25%,61669.000000,116665.000000
50%,124004.000000,142445.000000
75%,187036.000000,179444.000000
max,240760.000000,224428.000000


In [9]:
data_all = pd.concat([train, test])

In [7]:
h = data_all.sort_values('id').groupby('product_uid').apply(lambda df: len(df)).hist

In [29]:
data_all.sort_values('id').groupby('product_uid').apply(lambda df: len(df)).hist(bins=100)

In [30]:
from matplotlib import pyplot as plt
plt.show()

In [8]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor
from nltk.stem.snowball import SnowballStemmer

stemmer = SnowballStemmer('english')

df_train = pd.read_csv('./train.csv', encoding="ISO-8859-1")
df_test = pd.read_csv('./test.csv', encoding="ISO-8859-1")
# df_attr = pd.read_csv('../input/attributes.csv')
df_pro_desc = pd.read_csv('./product_descriptions.csv')

#df_train = df_train.head(1000)
#df_test = df_test.head(1000)
#df_pro_desc = df_pro_desc.head(1000)

num_train = df_train.shape[0]

def str_stemmer(s):
	return " ".join([stemmer.stem(word) for word in s.lower().split()])

def str_common_word(str1, str2):
	return sum(int(str2.find(word)>=0) for word in str1.split())


df_all = pd.concat((df_train, df_test), axis=0, ignore_index=True)

df_all = pd.merge(df_all, df_pro_desc, how='left', on='product_uid')

df_all['search_term'] = df_all['search_term'].map(lambda x:str_stemmer(x), n_jobs=4)
df_all['product_title'] = df_all['product_title'].map(lambda x:str_stemmer(x))
df_all['product_description'] = df_all['product_description'].map(lambda x:str_stemmer(x))

df_all['len_of_query'] = df_all['search_term'].map(lambda x:len(x.split())).astype(np.int64)

df_all['product_info'] = df_all['search_term']+"\t"+df_all['product_title']+"\t"+df_all['product_description']

df_all['word_in_title'] = df_all['product_info'].map(lambda x:str_common_word(x.split('\t')[0],x.split('\t')[1]))
df_all['word_in_description'] = df_all['product_info'].map(lambda x:str_common_word(x.split('\t')[0],x.split('\t')[2]))

df_all = df_all.drop(['search_term','product_title','product_description','product_info'],axis=1)

df_train = df_all.iloc[:num_train]
df_test = df_all.iloc[num_train:]
id_test = df_test['id']

y_train = df_train['relevance'].values
X_train = df_train.drop(['id','relevance'],axis=1).values
X_test = df_test.drop(['id','relevance'],axis=1).values

rf = RandomForestRegressor(n_estimators=15, max_depth=6, random_state=0, n_jobs=4)
clf = BaggingRegressor(rf, n_estimators=45, max_samples=0.1, random_state=25)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

pd.DataFrame({"id": id_test, "relevance": y_pred}).to_csv('submission.csv',index=False)


TypeError: map() got an unexpected keyword argument 'n_jobs'

In [9]:
df_all.head()

,id,product_uid,relevance,len_of_query,word_in_title,word_in_description
0,2,100001,3.00,2,1,1
1,3,100001,2.50,2,1,1
2,9,100002,3.00,2,1,1
3,16,100005,2.33,3,1,1
4,17,100005,2.67,3,3,2


In [26]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer('english')

def str_stemmer(s):
    return " ".join([stemmer.stem(i) for i in s.split()])

def f(s):
    title = [i.lower() for i in s['product_title'].split()]
    desc = [i.lower() for i in s['product_description'].split()]
    search = [i.lower() for i in s['search_term'].split()]
    s['title_search_common'] = len(set(title) & set(search)) 
    s['desc_search_common'] = len(set(desc) & set(search)) 
    return s

L = 10000
train = pd.read_csv('train.csv', encoding='ISO-8859-1').head(L)
test = pd.read_csv('test.csv', encoding='ISO-8859-1').head(L)
product_descriptions = pd.read_csv('product_descriptions.csv', encoding='ISO-8859-1').head(L)

df_all = pd.concat([train, test], axis=0)

df_all = pd.merge(df_all, product_descriptions, how='left', on='product_uid')
df_all.search_term = df_all.search_term.map(str_stemmer)
df_all.product_title = df_all.product_title.map(str_stemmer)
df_all.product_description = df_all.product_description.map(str_stemmer)

df_all = df_all.apply(f, axis=1)
df_all = df_all.drop(['product_title', 'product_description', 'search_term'], axis=1)


In [19]:
df_all.head()


,id,product_uid,relevance,title_search_common,desc_search_common
0,2,100001,3.00,1,1
1,3,100001,2.50,0,0
2,9,100002,3.00,0,0
3,16,100005,2.33,1,1
4,17,100005,2.67,3,2


In [71]:
#df_all.product_uid -= 100000 
df_all['product_uid'].head()

0    1
1    1
2    2
3    5
4    5
Name: product_uid, dtype: int64

In [27]:
n_train = train.shape[0]
df_all.product_uid -= 100000 
df_all_train = df_all.iloc[:n_train]
df_all_test = df_all.iloc[n_train:]
X_train = df_all_train.drop(['id', 'relevance'], axis=1).values
y_train = df_all_train['relevance'].values

X_test = df_all_test.drop(['id', 'relevance'], axis=1).values

rf = RandomForestRegressor(n_estimators=15, max_depth=6, random_state=0, n_jobs=4)
clf = BaggingRegressor(rf, n_estimators=45, max_samples=0.1, random_state=25)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)


In [ ]:
from sklearn.cross_validation import KFold
permutation = np.random.permutation(n_train)
X = X_train[permutation]
y = y_train[permutation]

n_cv = 5
kfold = KFold(n_train, n_cv)
rmse = np.zeros(n_cv)
for i, (tr_index, cv_index) in enumerate(kfold):
    X_tr, y_tr = X[tr_index, :], y[tr_index]
    X_cv, y_cv = X[cv_index, :], y[cv_index]
    
    clf.fit(X_tr, y_tr)
    y_pred = clf.predict(X_cv)
    rmse[i] = np.mean((y_pred - y_cv)**2)**0.5
    
    print rmse

[ 0.50528389  0.          0.          0.          0.        ]
[ 0.50528389  0.50967967  0.          0.          0.        ]
[ 0.50528389  0.50967967  0.5085029   0.          0.        ]


### Base-line performance

In [34]:
rmse_const = np.mean((y_train - 2)**2)**0.5
print rmse_const

0.674730664784


In [55]:
(.51 - .43)/(0.67 - .43)

0.3333333333333333

In [51]:
np.random.permutation(10)

array([0, 2, 9, 6, 4, 1, 7, 8, 5, 3])

In [48]:
np.random.rand(10)* 2 + 1

array([ 2.56160403,  1.8347247 ,  1.15252678,  2.98829945,  2.92688548,
        1.50812486,  2.97405915,  2.77963147,  2.20392214,  2.65977184])

In [50]:
y_pred_rand = np.random.rand(len(y_train))* 2 + 1
rmse_rand = np.mean((y_train - y_pred_rand)**2)**0.5
print rmse_rand


0.889443479739


In [31]:
y_pred

array([ 2.16539827,  2.12867757,  2.39065267, ...,  2.39625823,
        2.51676498,  2.51472045])